In [ ]:
1) Universal Forwarder (UF) Troubleshooting (Push → 9997)
✅ A. Confirm UF is connected to Indexer/HF

Component: TcpInputProc

index=_internal sourcetype=splunkd component=TcpInputProc earliest=-24h
| search "Accepted connection" OR "Connection from"
| stats count by host

✅ B. Find UF errors
index=_internal sourcetype=splunkd component=TcpInputProc (ERROR OR WARN) earliest=-24h
| table _time host log_level message

Common UF issues you will see:

SSL handshake failed

Connection reset

Error parsing metadata

Broken pipe

Blocked=true

2) Syslog Troubleshooting (UDP/TCP → 514/custom)
✅ A. UDP syslog receiving proof

Component: UdpInputProc

index=_internal sourcetype=splunkd component=UdpInputProc earliest=-24h
| stats count by host log_level

✅ B. TCP syslog receiving proof

Component: TcpInputProc

index=_internal sourcetype=splunkd component=TcpInputProc ("514" OR "syslog") earliest=-24h
| table _time host message

✅ C. Syslog drops / queue problems
index=_internal source=*metrics.log group=queue earliest=-1h
| stats max(current_size) as cur max(max_size) as max by name
| eval pct=round(cur/max*100,2)
| sort - pct


📌 If UDP, high traffic can drop logs (UDP is unreliable).
👉 Architect suggestion: use TCP syslog if high volume.

3) JSON Troubleshooting (HEC / API / Logpush)
✅ A. Confirm JSON is coming via HEC

Component: HttpInputDataHandler / HttpInputDataHandlerSSL

index=_internal sourcetype=splunkd (component=HttpInputDataHandler OR component=HttpInputDataHandlerSSL) earliest=-24h
| stats count by host log_level

✅ B. HEC errors (token / payload / format)
index=_internal sourcetype=splunkd (component=HttpInputDataHandler OR component=HttpInputDataHandlerSSL) (ERROR OR WARN) earliest=-24h
| table _time host log_level message


Common JSON/HEC issues:

Invalid token

HEC is disabled

payload too large

Incorrect channel

timestamp parsing issue

4) XML Logs Troubleshooting (mostly UF/File monitor)

XML logs are usually collected by:
✅ UF File monitoring
✅ scripted/modular input
✅ syslog with XML payload

✅ A. Check XML parsing issues

Components:

DateParserVerbose

LineBreakingProcessor

RegexReplacement

AggregatorPipelineProcessor

Timestamp / date parsing failures
index=_internal sourcetype=splunkd component=DateParserVerbose earliest=-24h
| table _time host message

Wrong event breaking (XML multi-line)
index=_internal sourcetype=splunkd component=LineBreakingProcessor earliest=-24h
| table _time host message


📌 XML Common issue = multiline event breaking
Fix needed:

SHOULD_LINEMERGE=false

LINE_BREAKER

BREAK_ONLY_BEFORE

TIME_PREFIX, MAX_TIMESTAMP_LOOKAHEAD

5) Generic “Not Reporting” root cause checks (All log types)
✅ A. Is data coming but not indexed? (nullQueue / routing issue)

Component: AggregatorPipelineProcessor

index=_internal sourcetype=splunkd component=AggregatorPipelineProcessor (WARN OR ERROR) earliest=-24h
| table _time host log_level message

✅ B. Index blocked / pipeline blocked

Component: Metrics (queue metrics)

index=_internal source=*metrics.log group=queue earliest=-1h
| stats max(current_size) as cur max(max_size) as max by name
| eval pct=round(cur/max*100,2)
| sort - pct


If you see pct > 80% frequently → ingestion delay / drop risk.

✅ 6) Best “One SPL” to identify method + issue quickly

Run this to know UF/syslog/JSON and errors:

index=_internal sourcetype=splunkd earliest=-24h
| eval method=case(
    component="TcpInputProc","UF / TCP Syslog",
    component="UdpInputProc","UDP Syslog",
    component="HttpInputDataHandler" OR component="HttpInputDataHandlerSSL","JSON/HEC",
    component="LineBreakingProcessor","XML Breaking",
    component="DateParserVerbose","Timestamp Parsing",
    1=1,"Other"
)
| search (method!="Other")
| stats count by method component log_level host
| sort - count

✅ Splunk Architect Troubleshooting Workflow (Very Short)

Check receiver components (TcpInputProc / UdpInputProc / HttpInputDataHandler)

Check parsing issues (LineBreakingProcessor / DateParserVerbose)

Check routing/index issues (AggregatorPipelineProcessor)

Check queue pressure (metrics.log group=queue)

Confirm data exists in target index (index=... | stats count by sourcetype)